In [24]:
import pandas as pd
import numpy as np 
import plotly.offline as py
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from geopy.geocoders import Nominatim
import folium
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from folium.plugins import HeatMap
sns.set_style('white')

In [25]:
#after imputation
data_deu = pd.read_csv('C:/Users/a0095989/Desktop/important/Thesis/Model_Thesis/imputed_deu_data.csv', parse_dates=[0])
# Verify dataset loading
print(data_deu.head())
data_ind = data_deu[data_deu['Country']=='Germany']

          dt  AverageTemperature  AverageTemperatureUncertainty    City  \
0 1743-11-01               6.425                          1.628  Aachen   
1 1743-12-01               0.689                          1.821  Aachen   
2 1744-01-01              -0.673                          2.095  Aachen   
3 1744-02-01               0.762                          1.923  Aachen   
4 1744-03-01               3.754                          1.626  Aachen   

   Country Latitude Longitude  
0  Germany   50.63N     6.34E  
1  Germany   50.63N     6.34E  
2  Germany   50.63N     6.34E  
3  Germany   50.63N     6.34E  
4  Germany   50.63N     6.34E  


In [26]:
mean_uncertainty = data_deu['AverageTemperatureUncertainty'].mean()
print(mean_uncertainty)

1.6144728825769268


In [27]:
# Adding Temperature Anomaly Feature

# Calculate the long-term mean temperature for each city
long_term_means = data_ind.groupby('City')['AverageTemperature'].mean()

# Add the anomaly feature by subtracting the long-term mean from each temperature value
data_ind['TemperatureAnomaly'] = data_ind.apply(
    lambda row: row['AverageTemperature'] - long_term_means[row['City']], axis=1
)

# Verify the added feature
#print(data_ind[['City', 'AverageTemperature', 'TemperatureAnomaly']].head(20))

# Extract Year and Month
data_ind['Year'] = data_ind['dt'].dt.year
data_ind['Month'] = data_ind['dt'].dt.month

# Add Seasons (example for northern hemisphere)
data_ind['Season'] = data_ind['Month'].apply(lambda x: 'Winter' if x in [12, 1, 2] 
                                      else 'Spring' if x in [3, 4, 5]
                                      else 'Summer' if x in [6, 7, 8]
                                      else 'Fall')

# Calculate Year-over-Year Temperature Changes
data_ind['TemperatureChange'] = data_ind.groupby('City')['AverageTemperature'].diff()


In [28]:
# Save the updated dataset to a CSV file
data_ind.to_csv('imputed_deu_data1.csv', index=False)


In [32]:
import pandas as pd

# Load the existing dataset
data_ind = pd.read_csv('C:/Users/a0095989/Desktop/important/Thesis/Model_Thesis/imputed_deu_data1.csv')

# Group by 'Year' and calculate the average temperature for each year
yearly_avg = data_ind.groupby('Year', as_index=False)['AverageTemperature'].mean()

# Rename the column to match your desired output
yearly_avg.rename(columns={'AverageTemperature': 'Yearly_Avg_Temperature'}, inplace=True)

# Merge the Yearly Average Temperature back into the original dataset
data_ind = data_ind.merge(yearly_avg, on='Year', how='left')

# Save the updated dataset, overwriting the original one
data_ind.to_csv('C:/Users/a0095989/Desktop/important/Thesis/Model_Thesis/imputed_deu_data1.csv', index=False)


Year
1744     8.294
1745     8.099
1746     8.489
1747     8.489
1748     8.489
         ...  
2008     9.836
2009     9.684
2010     8.332
2011    10.193
2012     9.531
Name: AverageTemperature, Length: 269, dtype: float64


In [30]:
long_term_means.head(20)

City
Aachen               8.823997
Augsburg             4.736021
Bergisch Gladbach    8.823997
Berlin               8.913005
Bielefeld            8.980344
Bochum               8.980344
Bonn                 8.823997
Bottrop              8.980344
Bremen               8.337281
Bremerhaven          8.337281
Brunswick            8.456355
Chemnitz             8.245591
Cologne              8.823997
Cottbus              8.913005
Darmstadt            8.173076
Dortmund             8.980344
Dresden              8.245591
Duisburg             8.823997
Düsseldorf           8.823997
Erfurt               7.596735
Name: AverageTemperature, dtype: float64